In [15]:
%load_ext dotenv
%dotenv ../.env -o


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
from langchain_community.document_loaders import PyPDFLoader  
from langchain_text_splitters import TokenTextSplitter
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate)
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import (ChatOpenAI,OpenAIEmbeddings)
from langchain_chroma.vectorstores import Chroma    

In [17]:
loader_pdf = PyPDFLoader("../Intro_to_AI_Transcript.pdf")
docs_list = loader_pdf.load()

In [18]:
token_splitter = TokenTextSplitter(encoding_name="cl100k_base", chunk_size = 200,chunk_overlap=40) 

In [19]:
docs_list_tokens_split = token_splitter.split_documents(docs_list)

In [20]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
retriever = vectorstore.as_retriever(search_type = 'mmr',
                                     search_kwargs = {'k':1,
                                                      'lambda_mult':0.7})

In [ ]:
vectorestore = Chroma.from_documents(documents=docs_list_tokens_split,embedding = embedding,
                                     persist_directory = "./intro-to-ai")

In [ ]:
retriever.invoke("What did Alan Turing do?")[0]

In [ ]:
print(retriever.invoke("What did Alan Turing do?")[0].page_content)

In [ ]:
PROMPT_RETRIEVING_S = '''You will receive a question from a student taking the Intro to AI course.
Answer the question using only the provided context.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_retrieving_s = SystemMessage(PROMPT_RETRIEVING_S)
prompt_template_retrieving_h = HumanMessagePromptTemplate.from_template(PROMPT_TEMPLATE_RETRIEVING_H)

chat_prompt_template_retrieving = ChatPromptTemplate([prompt_retrieving_s,
                                                      prompt_template_retrieving_h])

In [ ]:
chat = ChatOpenAI(model_name = "gpt-4o",
                  temperature = 0)

In [ ]:
str_output_parser = StrOutputParser()

In [ ]:
chain = (
    {'context': retriever,
     'question': RunnablePassthrough()}
    | chat_prompt_template_retrieving
    | chat
    | str_output_parser
)

In [ ]:
response = chain.stream('What did Alan Turing do?')

In [ ]:
for chunk in response:
    print(chunk, end = "")